# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,27.36,25,0,4.71,RU,1722412225
1,1,tamluk,22.3000,87.9167,34.33,58,100,3.54,IN,1722412226
2,2,ushuaia,-54.8000,-68.3000,2.81,56,0,3.09,AR,1722412228
3,3,yellowknife,62.4560,-114.3525,15.31,95,100,3.09,CA,1722412229
4,4,grand-bourg,15.8835,-61.3148,24.97,74,19,4.10,GP,1722412231


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display
# Configure the map plot
total_map_plot = city_data_df.hvplot.points('Lng',
                                            'Lat', 
                                            geo = True, 
                                            tiles = 'OSM',
                                            frame_width = 700,
                                            frame_height = 500,
                                            size = 'Humidity',
                                            color = 'City')

# Display the map
total_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp'] < 26) & (city_data_df['Max Temp'] > 15) & 
                            (city_data_df['Cloudiness'] != 0) & (city_data_df['Wind Speed'] < 5),:]
# Drop any rows with null values
ideal_df = ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,yellowknife,62.4560,-114.3525,15.31,95,100,3.09,CA,1722412229
4,4,grand-bourg,15.8835,-61.3148,24.97,74,19,4.10,GP,1722412231
5,5,puerto ayora,-0.7393,-90.3518,19.75,100,100,2.24,EC,1722412130
9,9,albany,42.6001,-73.9662,23.71,84,98,2.72,US,1722411982
13,13,kailua-kona,19.6406,-155.9956,24.22,99,18,1.21,US,1722412244
...,...,...,...,...,...,...,...,...,...,...
569,569,coahuayana de hidalgo,18.7000,-103.6583,24.29,94,100,1.08,MX,1722412983
572,572,lautoka,-17.6167,177.4667,22.92,78,20,2.06,FJ,1722412987
573,573,sao joao da barra,-21.6403,-41.0511,20.01,86,51,4.35,BR,1722412988
575,575,heilbad heiligenstadt,51.3833,10.1333,24.51,66,68,0.89,DE,1722412990


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.loc[:,["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,yellowknife,CA,62.4560,-114.3525,95,
4,grand-bourg,GP,15.8835,-61.3148,74,
5,puerto ayora,EC,-0.7393,-90.3518,100,
9,albany,US,42.6001,-73.9662,84,
13,kailua-kona,US,19.6406,-155.9956,99,
...,...,...,...,...,...,...
569,coahuayana de hidalgo,MX,18.7000,-103.6583,94,
572,lautoka,FJ,-17.6167,177.4667,78,
573,sao joao da barra,BR,-21.6403,-41.0511,86,
575,heilbad heiligenstadt,DE,51.3833,10.1333,66,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {"apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index,'Lat']
    lon = hotel_df.loc[index,'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    params["categories"] = "accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    # Convert the API response to JSON format
    name_address = name_address.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Remove rows with "No hotel found"
hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found",:]

# Display sample data
hotel_df

Starting hotel search
yellowknife - nearest hotel: No hotel found
grand-bourg - nearest hotel: Coco Beach Resort
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
albany - nearest hotel: No hotel found
kailua-kona - nearest hotel: Kona Seaside Hotel
nar'yan-mar - nearest hotel: Гостиница "Агат"
inirida - nearest hotel: Residencias El Turista del Guainia
kargil - nearest hotel: Border view
kilindoni - nearest hotel: Ibizza Inn
sao miguel do araguaia - nearest hotel: Hotel São Miguel
puerto baquerizo moreno - nearest hotel: Dory's House
namibe - nearest hotel: Nelsal Pensao
bayanhongor - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
kapchorwa - nearest hotel: Kingoo Cottages
hawaiian paradise park - nearest hotel: No hotel found
decorah - nearest hotel: Quality Inn & Suites Decorah
vorgashor - nearest hotel: София
camarinas - nearest hotel: O Parranda
mokhsogollokh - nearest hotel: No hotel found
belozersk - nearest hotel: Велес
thompson - 

,City,Country,Lat,Lng,Humidity,Hotel Name
4,grand-bourg,GP,15.8835,-61.3148,74,Coco Beach Resort
5,puerto ayora,EC,-0.7393,-90.3518,100,Hostal La Mirada De Solitario George
13,kailua-kona,US,19.6406,-155.9956,99,Kona Seaside Hotel
14,nar'yan-mar,RU,67.6713,53.0870,81,"Гостиница ""Агат"""
18,inirida,CO,3.8653,-67.9239,99,Residencias El Turista del Guainia
...,...,...,...,...,...,...
568,khilok,RU,51.3674,110.4676,44,Viola
572,lautoka,FJ,-17.6167,177.4667,78,Cathay Hotel
573,sao joao da barra,BR,-21.6403,-41.0511,86,Pousada Mediterrâneo
575,heilbad heiligenstadt,DE,51.3833,10.1333,66,Hotel Traube


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points('Lng',
                                        'Lat', 
                                        geo = True, 
                                        tiles = 'OSM',
                                        frame_width = 700,
                                        frame_height = 500,
                                        size = 'Humidity',
                                        color = 'City',
                                        hover_cols = ['Hotel Name','Country'])

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)